# **Course project on Technologies and Infrastructure for Big Data**
part I : image collecting from flickr


Project topic: image segmentation with data collection from www.flickr.com database 

Project executors:

Ma Cheng Yuan , ISU 372818 , J4133c

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b38d770ee0301d193e95ec9b528357e5b093e1b4c8971d0bb96046f9417de71c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
pip install flickrapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00


In [3]:
import pyspark.pandas as ps
import flickrapi
from flickrapi import FlickrAPI
import os
import warnings

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


#Download image from flick with flickrapi

- image segmentation , all_species = ['`**blue jay**', '**northern cardinal**', '**american goldfinch**`'] , each specie with 20 image 

Before we start parsing data, we have received key and secret code to connect to the API. You can get your own key in the Developers section of www.flickr.com or use ours.

In [4]:
from flickrapi import FlickrAPI
from pyspark.sql import SparkSession
from pyspark.sql.types import *

API_KEY = '513e24d8fc90dd4f767670d583ba1e41'
API_SECRET = '975b054fcd3c3371'
api = flickrapi.FlickrAPI(api_key=API_KEY, secret=API_SECRET, format='parsed-json')




In [5]:
import requests
import os
import sys

def create_folder(path):
    if not os.path.isdir(path):
        os.makedirs(path)

def download_images(urls, path):
    create_folder(path)  # makes sure path exists
    local_storage = []
    for url in urls:
        image_name = url.split("/")[-1]
        image_path = os.path.join(path, image_name)
        
        if not os.path.isfile(image_path):  # ignore if already downloaded
            response=requests.get(url,stream=True)

            with open(image_path,'wb') as outfile:
                outfile.write(response.content)
        local_storage.append(image_path)
    return local_storage

#explanation to argument api.photo.search method :


*   `text` : description of image
*   `sort` : flickr's built-in sorting method
*   `media`: data type (image,video)
*   `extras`: image metadata. We chose url_c, which will return url of the image thumbnail with a size of 800 pixels on the long side
*   `per_page`:number of images per query response page
*   `page`:page number of query responses







In [6]:
#from pyspark.sql.readwriter import DataFrameReader
import os
import time
import pandas

all_species = ['blue jay', 'northern cardinal', 'american goldfinch']
 
def download(urls,specie):
    fotos = []
    local_storage = []
    
    
    print('Getting urls for', specie)
    urls = urls

    print('Downlaing images for', specie)
    path = os.path.join('data', specie)
    
    
    local_storage = download_images(urls, path)
    
    return local_storage




df = ps.DataFrame()
print('df was created')
size = 20
flickr = FlickrAPI(API_KEY, API_SECRET)
for label in all_species :
  page = 1
  sum = []
  dataframe = ps.DataFrame()
  print(label)    
  while(len(sum) < size):
    photos = api.photos.search(text=label, sort='relevance', media='photos',
                  extras='url_c', per_page=size, page=page)['photos']['photo']
    i = -1
    while i < len(photos)-1:
      i = i+1     
      if 'url_c' in photos[i]:        
        if(photos[i]['url_c']=='None'):
         del photos[i]
         i = i-1
      else:
         del photos[i]
         i = i-1
    sum=sum+photos
    page = page + 1
  sum = sum[:20]
  dataframe = ps.DataFrame(sum)
  dataframe['label'] = [all_species.index(label)] * len(dataframe)
  dataframe = dataframe[dataframe['url_c']!='None']
  urls = dataframe['url_c'].tolist()
  local_storage = download(urls,label)
  print(len(dataframe))
  dataframe['root'] = local_storage
  df = ps.concat([df, dataframe], ignore_index=True)
 

df was created
blue jay
Getting urls for blue jay
Downlaing images for blue jay
20
northern cardinal
Getting urls for northern cardinal
Downlaing images for northern cardinal
20
american goldfinch
Getting urls for american goldfinch
Downlaing images for american goldfinch
20


In [13]:
df.head(5)

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,url_c,height_c,width_c,label,root
0,14143181441,75378295@N07,ffdc52e7c5,5546,6,Blue Jay,1,0,0,https://live.staticflickr.com/5546/14143181441...,800,599,0,data/blue jay/14143181441_ffdc52e7c5_c.jpg
1,22506516545,51102294@N05,fe40197b38,715,1,Blue jay,1,0,0,https://live.staticflickr.com/715/22506516545_...,533,800,0,data/blue jay/22506516545_fe40197b38_c.jpg
2,8695246698,95371549@N06,e949fbd20f,8540,9,Blue Jay,1,0,0,https://live.staticflickr.com/8540/8695246698_...,600,800,0,data/blue jay/8695246698_e949fbd20f_c.jpg
3,31618404466,24721968@N03,28bd67eef6,752,1,Blue Jay,1,0,0,https://live.staticflickr.com/752/31618404466_...,533,800,0,data/blue jay/31618404466_28bd67eef6_c.jpg
4,11124157635,84857817@N00,3c8aa26ddc,7381,8,Blue Jay,1,0,0,https://live.staticflickr.com/7381/11124157635...,535,800,0,data/blue jay/11124157635_3c8aa26ddc_c.jpg


In [8]:
from pyspark.sql.functions import udf, length, when, col, lit, broadcast,count,desc

modified_df = df.to_spark()
print('farm image of blue jay')
modified_df = modified_df.where(col('label')==0).groupBy('farm').agg(count("farm").alias("count")).select('farm','count').orderBy(desc(col('count')))
modified_df.show()

modified_df = df.to_spark()
print('farm image of northern cadrinal')
modified_df = modified_df.where(col('label')==1).groupBy('farm').agg(count("farm").alias("count")).select('farm','count').orderBy(desc(col('count')))
modified_df.show()

modified_df = df.to_spark()
print('farm image of goldfinch')
modified_df = modified_df.where(col('label')==2).groupBy('farm').agg(count("farm").alias("count")).select('farm','count').orderBy(desc(col('count')))
modified_df.show()

farm image of blue jay
+----+-----+
|farm|count|
+----+-----+
|   8|    7|
|   6|    4|
|   9|    4|
|   1|    2|
|   3|    1|
|   2|    1|
|   4|    1|
+----+-----+

farm image of northern cadrinal
+----+-----+
|farm|count|
+----+-----+
|  66|   19|
|   6|    1|
+----+-----+

farm image of goldfinch
+----+-----+
|farm|count|
+----+-----+
|  66|   16|
|   5|    2|
|   1|    1|
|   2|    1|
+----+-----+



In [9]:
modified_df = df.to_spark()
print('owner image of blue jay')
modified_df = modified_df.where(col('label')==0).groupBy('owner').agg(count("owner").alias("count")).select('owner','count').orderBy(desc(col('count')))
modified_df.show()

modified_df = df.to_spark()
print('owner image of northern cadrinal')
modified_df = modified_df.where(col('label')==1).groupBy('owner').agg(count("owner").alias("count")).select('owner','count').orderBy(desc(col('count')))
modified_df.show()

modified_df = df.to_spark()
print('owner image of goldfinch')
modified_df = modified_df.where(col('label')==2).groupBy('owner').agg(count("owner").alias("count")).select('owner','count').orderBy(desc(col('count')))
modified_df.show()


owner image of blue jay
+-------------+-----+
|        owner|count|
+-------------+-----+
| 24218402@N06|    2|
| 95988956@N00|    2|
| 24635080@N08|    2|
| 54526343@N03|    2|
| 75378295@N07|    1|
| 26027649@N03|    1|
|141746871@N07|    1|
| 12591684@N05|    1|
| 27956367@N06|    1|
| 95371549@N06|    1|
| 51102294@N05|    1|
| 84857817@N00|    1|
| 27254860@N04|    1|
| 24721968@N03|    1|
|119336417@N08|    1|
| 34020183@N03|    1|
+-------------+-----+

owner image of northern cadrinal
+-------------+-----+
|        owner|count|
+-------------+-----+
| 38802090@N06|   16|
| 75709980@N08|    2|
| 78998859@N04|    1|
|135168164@N03|    1|
+-------------+-----+

owner image of goldfinch
+-------------+-----+
|        owner|count|
+-------------+-----+
|129376752@N03|    3|
| 92361032@N05|    2|
| 93273765@N05|    2|
| 73683441@N07|    2|
| 23491871@N02|    1|
|146856323@N06|    1|
| 93058428@N06|    1|
|126304632@N05|    1|
|135842839@N08|    1|
| 41515458@N04|    1|
| 88978913@N08

In [10]:
print("there are total {} downloaded images".format(len(df)))


there are total 60 downloaded images


convert files with images into zip files and download it to create masked images (label images) using VGG image annotator 

https://annotate.officialstatistics.org/



In [11]:
!zip -r /content/data.zip /content/data

  adding: content/data/ (stored 0%)
  adding: content/data/blue jay/ (stored 0%)
  adding: content/data/blue jay/27654220552_69e94ece66_c.jpg (deflated 0%)
  adding: content/data/blue jay/26941886700_ffd2cc7d6c_c.jpg (deflated 0%)
  adding: content/data/blue jay/2334503592_614e71d8a4_c.jpg (deflated 0%)
  adding: content/data/blue jay/8695246698_e949fbd20f_c.jpg (deflated 0%)
  adding: content/data/blue jay/15882779474_87076fd9c8_c.jpg (deflated 1%)
  adding: content/data/blue jay/6591147097_f082483ac7_c.jpg (deflated 2%)
  adding: content/data/blue jay/7950900098_a911c5cf1d_c.jpg (deflated 1%)
  adding: content/data/blue jay/2333677567_3f05b28c50_c.jpg (deflated 0%)
  adding: content/data/blue jay/21992539444_13c1342bbf_c.jpg (deflated 0%)
  adding: content/data/blue jay/16324923976_ec53ecbd59_c.jpg (deflated 1%)
  adding: content/data/blue jay/31618404466_28bd67eef6_c.jpg (deflated 2%)
  adding: content/data/blue jay/22506516545_fe40197b38_c.jpg (deflated 1%)
  adding: content/data/b

In [12]:
from google.colab import files
uploaded = files.download('data.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>